In [9]:
!pip install datasets Transformers tensorflow tensorflow_datasets

In [10]:
#imports
import os
import fnmatch
import cv2
import numpy as np
import string
import time

from keras.preprocessing.sequence import pad_sequences

from keras.layers import Dense, LSTM, Reshape, BatchNormalization, Input, Conv2D, MaxPool2D, Lambda, Bidirectional
from keras.models import Model
from keras.activations import relu, sigmoid, softmax
import keras.backend as K
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
import tensorflow as tf

In [11]:
char_list = string.ascii_letters+string.digits


In [16]:
from datasets import load_dataset

ds = load_dataset("priyank-m/MJSynth_text_recognition",split="train")

Resolving data files:   0%|          | 0/25 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/25 [00:00<?, ?it/s]

In [17]:
def encode_to_labels(txt):
    # encoding each output word into digits
    dig_lst = []
    for index, char in enumerate(txt):
        try:
            dig_lst.append(char_list.index(char))
        except:
            print(char)

    return dig_lst

In [18]:
total_data=50000

In [19]:
ds2=ds.select(range(total_data))

In [20]:
imgarr=[]
orglabarr=[]
enclabarr=[]
length=[]

def preprocess(example):
    global imgarr,orglabarr, enclabarr,length
    for image,label in zip(example["image"],example["label"]):
      try:
        img= np.array((image))
      except OSError:
        continue
      img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
      w, h = img.shape
      #print(w,h)
      if h > 128 :

        img= cv2.resize(img,dsize=(128,w),interpolation=cv2.INTER_NEAREST)
        w,h =img.shape
      if w > 32:
        img= cv2.resize(img,dsize=(h,32),interpolation=cv2.INTER_NEAREST)
        w,h = img.shape
      if w < 32:
        add_zeros = np.ones((32-w, h))*255
        img = np.concatenate((img, add_zeros))

      if h < 128:
        add_zeros = np.ones((32, 128-h))*255
        img = np.concatenate((img, add_zeros), axis=1)
      img = np.expand_dims(img , axis = 2)

      img = img/255.0
      length.append(len(label))
      imgarr.append(img)
      orglabarr.append(label)
      enclabarr.append(encode_to_labels(label))
    examples=dict()

    return examples

ds2 = ds2.map(preprocess,batched=True,remove_columns=["image", "label"])


Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [21]:
processed_labels=pad_sequences(enclabarr,padding="post",maxlen=50)

In [23]:
inputs = Input(shape=(32,128,1))

conv1=Conv2D(64,(3,3), activation = 'relu', padding='same')(inputs)
pool1= MaxPool2D(pool_size=(2,2),strides=2)(conv1)

conv2=Conv2D(128,(3,3), activation = 'relu', padding='same')(pool1)
pool2= MaxPool2D(pool_size=(2,2),strides=2)(conv2)

conv3=Conv2D(256,(3,3), activation = 'relu', padding='same')(pool2)
conv4=Conv2D(256,(3,3), activation = 'relu', padding='same')(conv3)
pool4= MaxPool2D(pool_size=(2,1))(conv4)

conv5=Conv2D(512,(3,3), activation = 'relu', padding='same')(pool4)
batchnorm5 = BatchNormalization()(conv5)
conv6=Conv2D(512,(3,3), activation = 'relu', padding='same')(batchnorm5)
batchnorm6 = BatchNormalization()(conv6)

pool6=MaxPool2D(pool_size=(2,1))(batchnorm6)
conv7=Conv2D(512,(2,2), activation = 'relu')(pool6)

squeezed = Lambda(lambda x: K.squeeze(x, 1))(conv7)


# bidirectional LSTM layers with units=128
blstm1 = Bidirectional(LSTM(128, return_sequences=True, dropout = 0.2))(squeezed)
blstm2 = Bidirectional(LSTM(128, return_sequences=True, dropout = 0.2))(blstm1)

outputs = Dense(len(char_list)+1, activation = 'softmax')(blstm2)

act_model = Model(inputs, outputs)


In [1]:
import numpy as np
m=np.array([[1,2],[3,4],[5,6]])

In [3]:
m[0:,1:2]

array([[2],
       [4],
       [6]])

In [24]:
act_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 128, 1)]      0         
                                                                 
 conv2d (Conv2D)             (None, 32, 128, 64)       640       
                                                                 
 max_pooling2d (MaxPooling2  (None, 16, 64, 64)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 64, 128)       73856     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 8, 32, 128)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 8, 32, 256)        295168

In [25]:
labels = Input(name='the_labels', shape=[50], dtype='float32')
input_length = Input(name='input_length', shape=[1], dtype='int64')
label_length = Input(name='label_length', shape=[1], dtype='int64')


def ctc_lambda_func(args):
    y_pred, labels, input_length, label_length = args

    return K.ctc_batch_cost(labels, y_pred, input_length, label_length)


loss_out = Lambda(ctc_lambda_func, output_shape=(1,), name='ctc')([outputs, labels, input_length, label_length])

#model to be used at training time
model = Model(inputs=[inputs, labels, input_length, label_length], outputs=loss_out)

In [26]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 32, 128, 1)]         0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 32, 128, 64)          640       ['input_1[0][0]']             
                                                                                                  
 max_pooling2d (MaxPooling2  (None, 16, 64, 64)           0         ['conv2d[0][0]']              
 D)                                                                                               
                                                                                                  
 conv2d_1 (Conv2D)           (None, 16, 64, 128)          73856     ['max_pooling2d[0][0]'] 

In [27]:
model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer = 'adam')

filepath="best_model.hdf5"
checkpoint = ModelCheckpoint(
    filepath=filepath,
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    mode='auto',
    )
callbacks_list = [checkpoint]

In [28]:
shape=(32,128,1)
x=0

for i in imgarr:
  try:
    assert i.shape== shape
  except:
    j=i
    x+=1
print(x)

0


In [29]:
imgarr=np.array(imgarr)
processed_labels=np.array(processed_labels)
length=np.array(length)

In [30]:
batch_size = 64
epochs = 10
train_data=40000
model.fit(
    x=[
        np.array(imgarr[:train_data]),
        np.array(processed_labels[:train_data]),
        np.array([31]*train_data),
        np.array(length[:train_data]),
        ],
    y=np.zeros(train_data),
    batch_size=batch_size,
    epochs = epochs,
    validation_data = (
        [
          np.array(imgarr[train_data:]),
          np.array(processed_labels[train_data:]),
          np.array([31]*(total_data-train_data)),
          np.array(length[train_data:]),
        ],
        [np.zeros(total_data-train_data),])
        ,
    verbose = 1,
    callbacks = callbacks_list,
    )

Epoch 1/10
625/625 [==============================] - ETA: 0s - loss: 30.3892
Epoch 1: val_loss improved from inf to 29.57287, saving model to best_model.hdf5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


625/625 [==============================] - 85s 105ms/step - loss: 30.3892 - val_loss: 29.5729
Epoch 2/10
625/625 [==============================] - ETA: 0s - loss: 28.3767
Epoch 2: val_loss improved from 29.57287 to 27.64994, saving model to best_model.hdf5
625/625 [==============================] - 61s 97ms/step - loss: 28.3767 - val_loss: 27.6499
Epoch 3/10
625/625 [==============================] - ETA: 0s - loss: 25.9479
Epoch 3: val_loss improved from 27.64994 to 24.92974, saving model to best_model.hdf5
625/625 [==============================] - 59s 95ms/step - loss: 25.9479 - val_loss: 24.9297
Epoch 4/10
625/625 [==============================] - ETA: 0s - loss: 21.0350
Epoch 4: val_loss improved from 24.92974 to 16.09792, saving model to best_model.hdf5
625/625 [==============================] - 60s 96ms/step - loss: 21.0350 - val_loss: 16.0979
Epoch 5/10
625/625 [==============================] - ETA: 0s - loss: 11.3288
Epoch 5: val_loss improved from 16.09792 to 8.82721, savi

In [31]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [32]:
np.array(imgarr).shape

(50000, 32, 128, 1)

In [36]:
# load the saved best model weights
act_model.load_weights('/content/best_model.hdf5')

# predict outputs on validation images
imgs_np = tf.reshape(imgarr[0:10], [-1, 32, 128, 1])
prediction = act_model.predict(imgs_np)

# use CTC decoder
out = K.get_value(K.ctc_decode(prediction, input_length=np.ones(prediction.shape[0])*prediction.shape[1],
                         greedy=True)[0][0])

# see the results
i = 0
for x in out:
    print("original_text =  ", orglabarr[i])
    print("predicted text = ", end = '')
    for p in x:
        if int(p) != -1:
            print(char_list[int(p)], end = '')
    print('\n')
    i+=1

1/1 [==============================] - 5s 5s/step
original_text =   Lube
predicted text = Lube

original_text =   Spencerian
predicted text = Spencerian

original_text =   accommodatingly
predicted text = scanodatingly

original_text =   CARPENTER
predicted text = CARPENTER

original_text =   REGURGITATING
predicted text = HEREETAITES

original_text =   savannas
predicted text = savannas

original_text =   unfix
predicted text = unfix

original_text =   CAGOULES
predicted text = CAGOULES

original_text =   TRANSITS
predicted text = TRANITS

original_text =   looped
predicted text = looped

